In [ ]:
%pip install -r requirements.txt

Basic Azure OpenAI Integration

In [3]:
from openai import AzureOpenAI
import json

#Replace with your Pay-i API Key
payi_api_key="YOUR_PAYI_API_KEY"

#Replace with your Azure OpenAI Model Name, e.g. "gpt-4o-2024-05-13"
#Note that the full model name is required by Azure OpenAI
aoai_model = "YOUR_AZURE_OPENAI_MODEL"

#Replace with your Azure OpenAI Deployment Name, e.g. "test-4o"
aoai_deployment = "YOUR_AZURE_OPENAI_DEPLOYMENT"

#Replace with your deployed Azure OpenAI endpoint URI
aoai_instance_uri = "YOUR_AZURE_OPENAI_ENDPOINT"

aoai_key = "YOUR_AZURE_OPENAI_KEY"

aoai_key = "51a2f21381af4eff8aa9da08d1d51257"
aoai_api_version = "2024-02-15-preview"

#Replace with your Pay-i API Key
payi_api_key="bdfabb29ccf240a289586f47dc09b208"

#Replace with your Azure OpenAI Model Name, e.g. "gpt-4o-2024-05-13"
#Note that the full model name is required by Azure OpenAI
aoai_model = "gpt-4o-2024-05-13"

#Replace with your Azure OpenAI Deployment Name, e.g. "test-4o"
aoai_deployment = "testy-4o"

#Replace with your deployed Azure OpenAI endpoint URI
aoai_instance_uri = "https://openai-payi-qa-eastus.openai.azure.com"

#Replace with your Azure OpenAI Key
aoai_key = "51a2f21381af4eff8aa9da08d1d51257"
aoai_api_version = "2024-02-15-preview"

payi_headers = {
    "xProxy-Forward-api-key": aoai_key,
    "xProxy-Provider-BaseUri": aoai_instance_uri,
    "xProxy-RouteAs-Resource": aoai_model,
}

payi_base_url = "https://api.pay-i.com"
payi_aoai_url = payi_base_url + "/api/v1/proxy/azure.openai"

oai_client = AzureOpenAI(
    api_key=payi_api_key,
    api_version=aoai_api_version,
    azure_endpoint=payi_aoai_url,
    default_headers=payi_headers
)

response = oai_client.chat.completions.create(
    model=aoai_deployment,
    messages=[{"role": "user", "content": "Say 'this is a test'"}]
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

This is a test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": "0.00007"
        },
        "output": {
            "base": "0.000075"
        },
        "total": {
            "base": "0.000145"
        }
    },
    "request_id": "40006d65-0000-e000-b63f-84710c7967bb"
}


Handle streaming calls. xproxy_result is returned as part of the last chunk.

In [7]:
stream = oai_client.chat.completions.create(
    model=aoai_deployment,
    messages=[{"role": "user", "content": "tell me a very short story"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices is not None and len(chunk.choices) > 0 and chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
    if 'xproxy_result' in chunk.model_extra:
        print()
        print(json.dumps(chunk.model_extra['xproxy_result'], indent=4))

Once upon a time, in a bustling city, there was a tiny garden hidden between tall buildings. Every day, a little girl named Mia would visit the garden to water the flowers and talk to them. One morning, she found a beautiful golden key buried in the soil. Curious, she looked around and noticed an old, tiny door in the garden wall she had never seen before.

With a bit of courage, Mia used the key to unlock the door. As it creaked open, she discovered a magical world on the other side, filled with talking animals and glowing trees. The creatures welcomed her warmly, and Mia spent many joyful afternoons exploring this enchanting place. She kept the key with her always, knowing this secret world would be her special refuge whenever she needed a bit of magic in her life.

And so, the hidden garden and its magic door remained a cherished secret, forever giving Mia a place where her imagination could soar.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": "0.0

Use the Pay-i SDK to send a request with a request tag

In [9]:
from payi.lib.helpers import create_headers

response = oai_client.chat.completions.create(
    model=aoai_deployment,
    messages=[{"role": "user", "content": "Say 'this is a request tag test'"}],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

This is a request tag test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": "0.00008"
        },
        "output": {
            "base": "0.000105"
        },
        "total": {
            "base": "0.000185"
        }
    },
    "request_id": "40006d6e-0000-e000-b63f-84710c7967bb",
    "request_tags": [
        "x",
        "y"
    ]
}


Create a budget and make a request with that budget

In [10]:
from payi import Payi

payi_client = Payi(
    api_key=payi_api_key,
    base_url=payi_base_url
)

#Create a budget
budget_response = payi_client.budgets.create(
    #As long as the budget configuration remains the same across creates, the same budget name can be used repeatedly
    budget_name='Quickstart liberal allow budget', 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
response = oai_client.chat.completions.create(
    model=aoai_deployment, 
    messages=[{"role": "user", "content": "Say 'this is a budget test'"}],
    extra_headers=create_headers(
        request_tags=["x", "y"], 
        budget_ids=[budget_id]
    )
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Budget Created
Budget Name: Quickstart liberal allow budget
Budget ID: dab67602-3d41-465d-c073-08dcc629bb4c
This is a budget test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": "0.000075"
        },
        "output": {
            "base": "0.00009"
        },
        "total": {
            "base": "0.000165"
        }
    },
    "request_id": "40006d70-0000-e000-b63f-84710c7967bb",
    "request_tags": [
        "x",
        "y"
    ],
    "budgets": {
        "dab67602-3d41-465d-c073-08dcc629bb4c": {
            "state": "ok"
        }
    }
}


See budget status

In [11]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.output.base}")

Budget Name: Quickstart liberal allow budget
Budget ID: dab67602-3d41-465d-c073-08dcc629bb4c
Budget Creation Timestamp: 2024-08-26 23:49:31.786132+00:00
Budget Tags: ['example_budget']
Budget Input Base Cost: 0.00009
Budget Output Base Cost: 0.000108
Budget Total Base Cost: 0.000108


Make an ingest call with pre-computed token values

In [12]:
response = payi_client.ingest.units(
    category="system.openai",
    resource="gpt-4o",
    input=50,
    output=100,
    budget_ids= [budget_id],
    request_tags = ["a", "b"]
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.xproxy_result.cost.input.base}")
print(f"Output Base Cost: {response.xproxy_result.cost.output.base}")
print(f"Total Base Cost: {response.xproxy_result.cost.total.base}")

Ingest request ID: 40006d73-0000-e000-b63f-84710c7967bb
Input Base Cost: 0.00025
Output Base Cost: 0.0015
Total Base Cost: 0.00175


Reset a budget back to zero tracked cost

In [13]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.budget_reset_timestamp}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total.base}")

Budget has been successfully reset.
State prior to reset: 
Budget Name: Quickstart liberal allow budget
Budget ID: dab67602-3d41-465d-c073-08dcc629bb4c
Budget Tags: ['example_budget']
Budget Reset Timestamp: 2024-08-27 04:22:02.062598+00:00
Budget Input Base Cost: 0.00034
Budget Output Base Cost: 0.001608
Budget Total Base Cost: 0.001948

State after reset:
Budget Name: Quickstart liberal allow budget
Budget ID: dab67602-3d41-465d-c073-08dcc629bb4c
Budget Creation Timestamp: 2024-08-26 23:49:31.786132+00:00
Budget Tags: ['example_budget']
Budget Input Base Cost: CostData(input=CostDetails(base='0', overrun_base=0.0), output=CostDetails(base='0', overrun_base=0.0), total=CostDetails(base='0', overrun_base=0.0))
Budget Output Base Cost: 0
Budget Total Base Cost: 0


Create a small, conservative budget that will prevent calls from happening that exceed the maximum, then capture the output.

In [14]:
budget_response = payi_client.budgets.create(
    #As long as the budget configuration remains the same across creates, the same budget name can be used repeatedly
    budget_name='Quickstart conservative block budget',
    max=0.01, 
    budget_type="conservative",
    budget_response_type="block",
    budget_tags=["budget_block_example"]
)
cons_budget = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_response.budget.budget_name}")
print(f"Budget ID: {budget_response.budget.budget_id}")

try:
    response = oai_client.chat.completions.create(
        model=aoai_deployment,
        messages=[{"role": "user", "content": "tell me a short story about a toy"}],
        extra_headers=create_headers(
            request_tags=["toy_story"], 
            budget_ids=[cons_budget]
        )
    )
except Exception as e:
    print(json.dumps(e.body, indent=4))

Budget Created
Budget Name: Quickstart conservative block budget
Budget ID: 659c27eb-fde6-4941-c074-08dcc629bb4c
{
    "xproxy_result": {
        "blocked_budget_ids": [
            "659c27eb-fde6-4941-c074-08dcc629bb4c"
        ],
        "request_id": "40006d78-0000-e000-b63f-84710c7967bb",
        "request_tags": [
            "toy_story"
        ],
        "budgets": {
            "659c27eb-fde6-4941-c074-08dcc629bb4c": {
                "state": "blocked"
            }
        }
    },
    "message": "Insufficient budget for the requested operation",
    "statusCode": 400,
    "xproxy_error": {
        "code": "not_enough_budget",
        "message": "Insufficient budget for the requested operation"
    }
}


Create an experience type and instance, then send a request with the experience instance

In [15]:
# Create an experience type
exp_type_response = payi_client.experiences.types.create(
    name="A quickstart Experience",
    description="An example of an experience"
)

# Create an experience of the type
exp_response = payi_client.experiences.create(
    experience_type_id=exp_type_response.experience_type_id,
)

#Make a request using the new budget and request tags
response = oai_client.chat.completions.create(
    model=aoai_deployment, 
    messages=[{"role": "user", "content": "Say 'this is a budget test'"}],
    extra_headers=create_headers(
        request_tags=["x-experience", "y-experience"], 
        budget_ids=[budget_id],
        experience_id=exp_response.experience_id
    )
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

This is a budget test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": "0.000075"
        },
        "output": {
            "base": "0.00009"
        },
        "total": {
            "base": "0.000165"
        }
    },
    "request_id": "40006f1b-0000-f700-b63f-84710c7967bb",
    "request_tags": [
        "x-experience",
        "y-experience"
    ],
    "budgets": {
        "dab67602-3d41-465d-c073-08dcc629bb4c": {
            "state": "ok"
        }
    }
}


Send a request with a budget and user ID

In [16]:
#Make a request using the budget and user id
response = oai_client.chat.completions.create(
    model=aoai_deployment, 
    messages=[{"role": "user", "content": "Say 'this is a budget test'"}],
    extra_headers=create_headers(
        budget_ids=[budget_id],
        # user id can be any string value
        user_id="example_user_id"
    )
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

This is a budget test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": "0.000075"
        },
        "output": {
            "base": "0.00009"
        },
        "total": {
            "base": "0.000165"
        }
    },
    "request_id": "4000787c-0000-ec00-b63f-84710c7967bb",
    "budgets": {
        "dab67602-3d41-465d-c073-08dcc629bb4c": {
            "state": "ok"
        }
    }
}


List and then delete all budgets

In [ ]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)